In [350]:
# Создание базовой таблицы по населеным пунктам Республики Тыва, со стат данными оп демографии
# Автор Иргит В.
import vk_api
import pandas as pd
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim

In [360]:
vk_session = vk_api.VkApi( '***', '***' )
vk_session.auth()
vk = vk_session.get_api()

city = [i for i in vk.database.getCities( region_id=1157049, country_id=1, count=300 )["items"]]
df = json_normalize(city).fillna('город')

In [352]:
geolocator = Nominatim(user_agent="shaltay@inbox.ru")
name = df['title'] + ', Тыва'
#df['full_adress'] = name.apply(geolocator.geocode, timeout=100)
df['lat'] = name.apply(geolocator.geocode, timeout=100).apply(lambda x: (x.latitude) if x else None)
df['lot'] = name.apply(geolocator.geocode, timeout=100).apply(lambda x: (x.longitude) if x else None)
df['full_adress'] = name.apply(geolocator.geocode, timeout=100)

In [353]:
df.head()

,area,id,region,title,lat,lot,full_adress
0,Каа-Хемский район,1157087,Тыва,Авыйган,51.320499,95.592648,"(Авыйган, Усть-Буренский сумон, Каа-Хемский ко..."
1,Тоджинский район,1157181,Тыва,Адыр-Кежиг,52.449773,96.241468,"(Адыр-Кежиг, Тоджинский кожуун, Республика Тыв..."
2,Сут-Хольский район,1157148,Тыва,Ак-Даш,51.341095,91.128167,"(Ак-Даш, Сут-Хольский кожуун, Республика Тыва,..."
3,город,9863,Тыва,Ак-Довурак,51.174081,90.599991,"(Ак-Довурак, городской округ Ак-Довурак, Респу..."
4,Чаа-Хольский район,1157200,Тыва,Ак-Дуруг,51.399509,92.207482,"(Ак-Дуруг, Чаа-Хольский кожуун, Республика Тыв..."


In [354]:
file_csv = r'd:\Temp\Новая папка (15)\статистика\численость.csv'
temp_data = pd.read_csv(file_csv, encoding='cp1251', sep=';')
temp_data['Unnamed: 0'].replace('\n','', regex=True)
temp_data.head()

,Unnamed: 0,Всего,Мужчины,Женщины
0,Городской округ г. Кызыл - городское население...,109918,51152,58766
1,Городской округ г. Ак-Довурак - городское насе...,13468,6199,7269
2,Муниципальный район Бай-Тайгинский кожуун -\nс...,10803,5184,5619
3,Сельское поселение сумон Бай-Тал,1786,848,938
4,Сельское поселение сумон Кара-Холь,1395,669,726


In [355]:
def in_pop(temp_data, x):
       for i in temp_data['Unnamed: 0']:
            arr = i.split(' ')
            if x in arr:
                data = temp_data.loc[temp_data['Unnamed: 0'] == i]
                return data['Всего'].values[0]
def man(temp_data, x):
       for i in temp_data['Unnamed: 0']:
            arr = i.split(' ')
            if x in arr:
                data = temp_data.loc[temp_data['Unnamed: 0'] == i]
                return data['Мужчины'].values[0]
def female(temp_data, x):
       for i in temp_data['Unnamed: 0']:
            arr = i.split(' ')
            if x in arr:
                data = temp_data.loc[temp_data['Unnamed: 0'] == i]
                return data['Женщины'].values[0]

df['population_2010'] = df['title'].apply(lambda x: (in_pop(temp_data, x)))
df['man_2010'] = df['title'].apply(lambda x: (man(temp_data, x)))
df['female_2010'] = df['title'].apply(lambda x: (female(temp_data, x)))

,area,id,region,title,lat,lot,full_adress,population_2010,man_2010,female_2010
0,Каа-Хемский район,1157087,Тыва,Авыйган,51.320499,95.592648,"(Авыйган, Усть-Буренский сумон, Каа-Хемский ко...",NaN,NaN,None
1,Тоджинский район,1157181,Тыва,Адыр-Кежиг,52.449773,96.241468,"(Адыр-Кежиг, Тоджинский кожуун, Республика Тыв...",NaN,NaN,None
2,Сут-Хольский район,1157148,Тыва,Ак-Даш,51.341095,91.128167,"(Ак-Даш, Сут-Хольский кожуун, Республика Тыва,...",563.0,254.0,309
3,город,9863,Тыва,Ак-Довурак,51.174081,90.599991,"(Ак-Довурак, городской округ Ак-Довурак, Респу...",13468.0,6199.0,7269
4,Чаа-Хольский район,1157200,Тыва,Ак-Дуруг,51.399509,92.207482,"(Ак-Дуруг, Чаа-Хольский кожуун, Республика Тыв...",1408.0,697.0,711
5,Чеди-Хольский район,1157206,Тыва,Ак-Тал,51.174322,93.407191,"(Ак-Тал, Чеди-Хольский кожуун, Республика Тыва...",NaN,NaN,None
6,Тес-Хемский район,1157173,Тыва,Ак-Эрик,50.514201,94.615283,"(Ак-Эрик, Тес-Хемский кожуун, Республика Тыва,...",NaN,NaN,None
7,Барун-Хемчикский район,1157063,Тыва,Аксы-Барлык,51.117568,90.402182,"(Аксы-Барлык, Барун-Хемчикский кожуун, Республ...",934.0,457.0,477
8,Сут-Хольский район,1157149,Тыва,Алдан-Маадыр,51.296580,90.975163,"(Алдан-Маадыр, Сут-Хольский кожуун, Республика...",1109.0,555.0,554
9,Пий-Хемский район,1157131,Тыва,Аржаан,52.059244,93.605353,"(Аржаан, Пий-Хемский кожуун, Республика Тыва, ...",835.0,417.0,418


In [358]:
df.to_excel('populat_tuva.xlsx', engine='xlsxwriter')